In [52]:
# Many-to-one LSTM network (Divides the network into n layers evenly starting from the input dimension)
# i.e. a 300 length input vector with 2 levels is split into a 150 node layer and a 1 node layer
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from math import *
import time
import re
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [53]:
def fullprint(*args, **kwargs):
  from pprint import pprint
  import numpy
  opt = numpy.get_printoptions()
  numpy.set_printoptions(threshold='nan')
  pprint(*args, **kwargs)
  numpy.set_printoptions(**opt)

In [54]:
NUM_LAYERS = 2
VEC_SIZE = 300
hidden_layer_sizes = [int(VEC_SIZE - i * VEC_SIZE/NUM_LAYERS) for i in range(NUM_LAYERS)] + [1]
print(hidden_layer_sizes)

[300, 150, 1]


In [55]:
# Make headline embeddings
print("Loading model...")
start = time.time()
model = Doc2Vec.load('NYTimesYearDocVecs.bin')
model.delete_temporary_training_data(keep_doctags_vectors=True, 
                                     keep_inference=True)
print("Model loaded in {}s".format(time.time()-start))

Loading model...
Model loaded in 426.732879877s


In [56]:
dv = model.docvecs
doc_tags = dv.offset2doctag
from collections import defaultdict
daily_headline = defaultdict(list)
for tag in doc_tags:
    date_str = re.match('[0-9]{4}-[0-9]{2}-[0-9]{2}', tag).group()
    daily_headline[date_str].append(dv[tag])

In [57]:
# print(daily_headline.keys())
# print([daily_headline[k] for k in daily_headline.keys()[0]])
assert len(daily_headline.keys()) == len(set(daily_headline.keys()))

## Multiple ways to go about prediction:
  1. Input document vectors for each day into a variable length sequence, many-to-one layer. This feeds into a layer that takes 20 of these many-to-one outputs (20 days worth of data) and predicts probabilities `[p_up, p_down]` for day 21. Could also predict ticker value on day 21.
  2. Input fixed sequence of averaged daily doc vectors to model. Proceed similarly to above.
  3. Cutoff the amount of headlines used based on a metric or just based on a hard-coded
  number.

### The above list goes from most difficult to least so let's go with #3 first

In [61]:
def length(sequence):
    used = tf.sign(tf.reduce_max(tf.abs(sequence), reduction_indices=2))
    length = tf.reduce_sum(used, reduction_indices=1)
    length = tf.cast(length, tf.int32)
    return length

In [62]:
print(length(daily_headline))

KeyboardInterrupt: 

In [ ]:
def batch_producer(raw_data, batch_size, num_steps):
    raw_data = tf.convert_to_tensor(raw_data, name="raw_data", dtype=tf.int32)

    data_len = tf.size(raw_data)
    batch_len = data_len // batch_size
    data = tf.reshape(raw_data[0: batch_size * batch_len],
                      [batch_size, batch_len])

    epoch_size = (batch_len - 1) // num_steps
    
    i = tf.train.range_input_producer(epoch_size, shuffle=False).dequeue()
    x = data[:, i * num_steps:(i + 1) * num_steps]
    x.set_shape([batch_size, num_steps])
    y = data[:, i * num_steps + 1: (i + 1) * num_steps + 1]
    y.set_shape([batch_size, num_steps])
    return x, y

In [11]:
# Placeholder, variable batch size (None)
with tf.name_scope('input'):
    X = tf.placeholder(tf.float32, [None, 300])
    Y = tf.placeholder(tf.float64, [None, 1])

In [10]:
def lstm_cell(size):
  return tf.contrib.rnn.BasicLSTMCell(size)

In [18]:
# Normalized sample to initialize the weight matrices
with tf.name_scope('network'):

    state_layers = []
    multiple_cells = []
    # Initial state of the LSTM memory.
    for size in hidden_layer_sizes[1:]:
#         h_i = tf.zeros([current_batch_size, size])
#         c_i = tf.zeros([current_batch_size, size])
#         state = h_i, c_i
#         print(state)
#         state_layers.append(state)
        multiple_cells.append(lstm_cell(size))
    stacked_lstm = tf.contrib.rnn.MultiRNNCell(multiple_cells)
    print(state_layers)
    
    initial_state = state = stacked_lstm.zero_state(batch_size, tf.float32)
#     for i in range(num_steps):
#         # The value of state is updated after processing each batch of words.
#         output, state = stacked_lstm(words[:, i], state)

#         # The rest of the code.
#         # ...

#     final_state = state
#         W1 = tf.Variable(tf.truncated_normal([784, 256], stddev=0.1))
#         b1 = tf.Variable(tf.constant(0.1, shape = [1, 256]))
#         layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)

NameError: name 'current_batch_size' is not defined

In [ ]:
# Network 
